In [2]:
# import modules

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os
import urllib

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [13]:
# examine if datasets exist

BOSTON_PATH = {}
BOSTON_URL = {}

BOSTON_PATH['TRAIN'] = 'boston_train.csv'
BOSTON_URL['TRAIN'] = 'http://download.tensorflow.org/data/boston_train.csv'

BOSTON_PATH['TEST'] = 'boston_test.csv'
BOSTON_URL['TEST'] = 'http://download.tensorflow.org/data/boston_test.csv'

BOSTON_PATH['PREDICT'] = 'boston_predict.csv'
BOSTON_URL['PREDICT'] = 'http://download.tensorflow.org/data/boston_predict.csv'

for dataname in ['TRAIN', 'TEST', 'PREDICT']:
    if not os.path.exists(BOSTON_PATH[dataname]):
        raw = urllib.urlopen(BOSTON_URL[dataname]).read()
        with open(BOSTON_PATH[dataname], 'w') as f:
            f.write(raw)

print('Datasets are ready.')

Datasets are ready.


In [14]:
# read data from datasets

COLUMNS = ['crim', 'zn', 'indus', 'nox', 'rm', 'age', 'dis', 'tax', 'ptratio', 'medv']
FEATURES = COLUMNS[:-1]
LABEL = COLUMNS[-1]

[training_set, test_set, prediction_set]  = [
    pd.read_csv(
        BOSTON_PATH[dataname], 
        skipinitialspace=True, 
        skiprows=1, 
        names=COLUMNS
    ) for dataname in ['TRAIN', 'TEST', 'PREDICT']
]

print('Readidng from datasets successfully.')

Readidng from datasets successfully.


In [16]:
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

print('Feature columns created.')

Feature columns created.


In [17]:
regressor = tf.contrib.learn.DNNRegressor(
    feature_columns=feature_cols, 
    hidden_units=[10, 10], 
    model_dir='/tmp/boston_model'
)

print('Model created.')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f75cea73190>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Model created.


In [19]:
def my_input_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values)
                   for k in FEATURES}
    labels = tf.constant(data_set[LABEL].values)
    
    return feature_cols, labels

In [20]:
stepsNow = 5000
regressor.fit(input_fn=lambda: my_input_fn(training_set), steps=stepsNow)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/boston_model/model.ckpt.
INFO:tensorflow:loss = 22421.2, step = 1
INFO:tensorflow:global_step/sec: 1569.27
INFO:tensorflow:loss = 101.081, step = 101
INFO:tensorflow:global_step/sec: 1692.45
INFO:tensorflow:loss = 86.4818, step = 201
INFO:tensorflow:global_step/sec: 1681.58
INFO:tensorflow:loss = 83.0368, step = 301
INFO:tensorflow:global_step/sec: 1558.7
INFO:tensorflow:loss = 81.087, step = 401
INFO:tensorflow:global_step/sec: 1704.95
INFO:tensorflow:loss = 79.5846, step = 501
INFO:tensorflow:global_step/sec: 1648.96
INFO:tensorflow:loss = 78.2834, step = 601
INFO:ten

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7f75cea73110>, 'hidden_units': [10, 10], 'feature_columns': (_RealValuedColumn(column_name='crim', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='zn', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='indus', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='nox', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='rm', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='age', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='dis', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='tax', dimension=1, default_value=None, dtype=tf.float

In [21]:
stepsNow = 1
ev = regressor.evaluate(input_fn=lambda: my_input_fn(test_set), steps=stepsNow)
loss_score = ev['loss']
print('Loss: {0:f}'.format(loss_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-04-23-10:43:49
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-04-23-10:43:49
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 16.8024
Loss: 16.802423


In [22]:
# make predictions

y = regressor.predict(input_fn=lambda: my_input_fn(prediction_set))
predictions = list(itertools.islice(y, 6))
print('Predictions: {}'.format(str(predictions)))

Predictions: [33.710159, 18.039553, 22.329962, 33.062084, 12.897905, 18.934099]
